### 单调栈结构monotonousStack

#### 生成arr(无重复值)中的每个值左右两边离自己最近且比自己小的索引列表
使用单调栈结构，栈中储存由小到大数组，遍历arr中所有值，每次尝试压入栈：
1. 栈空或栈顶比当前数小，可以压入，压入在arr的位置
2. 栈顶比当前数大，此时弹出栈顶，记录信息对于栈顶数(左侧最近为栈顶下面的数，右侧最近为当前数）。弹到满足条件的栈顶时，压入当前数
3. 重复直到所有数压入栈后，若栈中仍有数，依次弹出，弹出值的没有右侧，左侧是下面压着的，若下面没压则为空。

    时间复杂度O(N)，因为每个数最多进一次出一次栈

In [6]:
def getNearLessNoRepeat(arr):
    res = [[-1] * 2 for _ in range(len(arr))]
    stack = list()
    for i in range(len(arr)):
        while stack and arr[stack[-1]] > arr[i]: #当前数落不上
            j = stack.pop()
            leftLessIndex = -1 if not stack else stack[-1]
            res[j][0] = leftLessIndex
            res[j][1] = i
        stack.append(i)
    while stack:
        j = stack.pop()
        leftLessIndex = -1 if not stack else stack[-1]
        res[j][0] = leftLessIndex
        res[j][1] = -1
    return res

#### 生成arr(有重复值)中的每个值左右两边离自己最近且比自己小的索引列表
使用单调栈结构，栈中储存由小到大数组，遍历arr中所有值，每次尝试压入栈：压入索引(链表或列表储存)，索引指向值
1. 栈空或栈顶比当前数小，可以压入
2. 栈顶比当前数大，此时弹出栈顶，记录信息对于栈顶数(左侧最近为栈顶下面的索引中的最后一个位置，右侧最近为当前数）
3. 栈顶与当前数相同时，把当前数压入栈顶索引列表中的最后位置
3. 重复直到所有数压入栈后，若栈中仍有数，依次弹出，弹出值的没有右侧，左侧是下面压着的索引最后一个位置，若下面没压则为空。

In [7]:
def getNearLessWithRepeat(arr):
    res = [[-1] * 2 for _ in range(len(arr))]
    stack = list()
    for i in range(len(arr)):
        while stack and arr[stack[-1][0]] > arr[i]: #当前数落不上
            js = stack.pop()
            leftLessIndex = -1 if not stack else stack[-1][-1] #最后一个位置
            for each in js:
                res[each][0] = leftLessIndex
                res[each][1] = i
        #当前数 与栈顶相等
        if stack and arr[stack[-1][0]] == arr[i]:
            stack[-1].append(i)
        # 可以压入，作为一个新列表头部
        else:
            stack.append([i])
    while stack:
        js = stack.pop()
        leftLessIndex = -1 if not stack else stack[-1]
        for each in js:
            res[each][0] = leftLessIndex
            res[each][1] = -1
    return res

### 单调栈实际应用

#### <span class="mark">正数</span>数组中子数组累加和乘最小值的最大值
给定一个只包含正数的数组arr，arr中任何一个子数组sub。一定可以计算出A = (sub累加和)*(sub中最小值)。求所有sub中的A最大值


In [10]:
'''
思路：
从0位置开始，以当前位置做最小值时，满足条件的子数组都有哪些，计算出最大值A？
1.因为当前位置相同，那么A最大，即累加和最大。那么就是找到向左、向右找到最近的比当前数小的两个位置，两位置间所有值累加和一定最大。
2.剩下的问题就是 求i...j的累加和最大的那个值即A最大值。（求i...j的累加和 使用一个预处理的前缀和数组，O(1)计算出）
3.可以使用无重复值算法，当栈弹出时，相等也弹出。虽然弹出的相等值会计算错，但有招一日当前数(或最后一个相等数)可以算对。
'''
def getPreSum(arr):
    preSum = [None] * len(arr)
    pre = 0
    for i in range(len(arr)):
        pre += arr[i]
        preSum[i] = pre
    return preSum
def getCertainSum(arr, i, j):
    if i == -1:
        return arr[j]
    else:
        return arr[j] - arr[i]
def maxSubSumMultiMin(arr):
    if not arr:
        return 0
    stack = list()
    N = len(arr)
    rangeList = [[None] * 2 for _ in range(N)]
    for index in range(N):
        while stack and arr[stack[-1]] >= arr[index]:
            popIndex = stack.pop()
            rangeList[popIndex][1] = index
            if stack:
                rangeList[popIndex][0] = stack[-1]
            else:
                rangeList[popIndex][0] = -1
        stack.append(index)
    while stack:
        popIndex = stack.pop()
        rangeList[popIndex][1] = N
        if stack:
            rangeList[popIndex][0] = stack[-1]
        else:
            rangeList[popIndex][0] = -1
    preSum = getPreSum(arr)
    maxAns = [None] * N
    for i in range(N):
        rangeL, rangeR = rangeList[i][0], rangeList[i][1] - 1
        maxAns[i] = arr[i] * getCertainSum(preSum, rangeL, rangeR)
    ans = 0
    for i in range(N):
        ans = max(maxAns[i], ans)
    return ans
        

In [11]:
def maxSubSumMultiMin_test(arr):
    if not arr:
        return 0
    N = len(arr)
    ans = 0
    for i in range(N):
        for j in range(i, N):
            sub = arr[i:j+1]
            min_ = min(sub)
            sum_ = sum(sub)
            ans = max(min_ * sum_, ans)
    return ans

In [14]:
import random
def generateRandomPosArr(maxLen, maxValue):
    arr = list()
    for i in range(maxLen):
        arr.append(random.randint(1, maxValue))
    return arr

testTime = 10000
maxValue = 100
maxLen = 100
succeed=True
for _ in range(testTime):
    value = random.randint(1, maxValue)
    length = random.randint(0, maxLen)
    arr = generateRandomPosArr(length, value)
    ans1 = maxSubSumMultiMin_test(arr)
    ans2 = maxSubSumMultiMin(arr)
    if ans1 != ans2:
        print(n, ans1, ans2)
        print("something wrong.")
        succeed = False
        break
if succeed:
    print("succeed.")

succeed.


#### 柱状图中最大的矩形
非负数组，代表直方图。返回拼成的长方形最大面积。
https://leetcode.cn/problems/largest-rectangle-in-histogram/

In [19]:
'''
1.从0位置开始，必须以当前位置为高时，能组成的最大长方形。
2.找到比当前位置小的左右位置。长(i...j多少个数)*高即长方形面积。
3.仍然可以忽略相等值的错误信息。
'''

def largestRectangleArea(heights) -> int:
    if not heights:
        return 0

    stack = list()
    N = len(heights)
    rangeList = [[None] * 2 for _ in range(N)]
    for index in range(N):
        while stack and heights[stack[-1]] >= heights[index]:
            popIndex = stack.pop()
            rangeList[popIndex][1] = index
            if stack:
                rangeList[popIndex][0] = stack[-1]
            else:
                rangeList[popIndex][0] = -1
        stack.append(index)
    while stack:
        popIndex = stack.pop()
        rangeList[popIndex][1] = N
        if stack:
            rangeList[popIndex][0] = stack[-1]
        else:
            rangeList[popIndex][0] = -1
    ans = 0
    for i in range(N):
        rangeL, rangeR = rangeList[i][0], rangeList[i][1] - 1
        ans = max((rangeR - rangeL) * heights[i], ans)
    return ans

#### 全是1的最大子矩形面积
二维数组matrix，不是0就是1. 返回最大子矩形面积。
https://leetcode.cn/problems/maximal-rectangle/

In [29]:
'''
暴力解，遍历所有子矩阵为N^4。 随便点一个点，有N^2个点。点两个点就可以形成一个矩形。每个子矩阵查看是否是矩形，n^2。
所以暴力解的总代价O(N^6)


最优解O(N^2)：
[[1,1,1,1,1]   index = 0
 [1,0,1,1,1]   index = 1
 [1,1,1,0,1]   index = 2
 [1,1,1,1,1]]  index = 3
从上至下
先求第0行做地基，求哪个子矩阵1最多。[1,1,1,1,1], 这个list用单调栈处理一遍。O(N)
再求第一行做地基，求哪个子矩阵1最多 [2,0,2,2,2]
再求第二行做地基，求哪个子矩阵1最多 [3,1,3,0,3]
再求第三行做地基，求哪个子矩阵1最多 [4,2,4,1,4], 一共N行，所以说N*N
'''
def maximalRectangle(arr):
    if not arr or not arr[0]:
        return 0
    N = len(arr)
    M = len(arr[0])
    base = [int(i) for i in arr[0]]
    print(base)
    maxAns = getMaxRect(base)
    for i in range(1, N):
        base = getBase(base, arr[i])
        print(base)
        maxAns = max(maxAns, getMaxRect(base))
    return maxAns
        
def getMaxRect(arr):
    N = len(arr)
    stack = list()
    rangeList = [[None]*2 for _ in range(N)]
    for i in range(N):
        while stack and arr[stack[-1]] >= arr[i]:
            popIndex = stack.pop()
            if stack:
                rangeList[popIndex][0] = stack[-1]
            else:
                rangeList[popIndex][0] = -1
            rangeList[popIndex][1] = i
        stack.append(i)
    while stack:
        popIndex = stack.pop()
        if stack:
            rangeList[popIndex][0] = stack[-1]
        else:
            rangeList[popIndex][0] = -1
        rangeList[popIndex][1] = N
    ans = 0
    for i in range(N):
        rangeL,rangeR = rangeList[i][0], rangeList[i][1] - 1
        ans = max(arr[i] * (rangeR - rangeL), ans)
    return ans
        
def getBase(base, arr):
    for i in range(len(base)):
        if int(arr[i]) == 0:
            base[i] = 0
        else:
            base[i] += int(arr[i])
    return base
            
        

In [30]:
# arr = [["1","0","1","0","0"],["1","0","1","1","1"],["1","1","1","1","1"],["1","0","0","1","0"]]
arr = [["0","1"],["1","0"]]
maximalRectangle(arr)

[0, 1]
[1, 0]


1

#### 全是1的子矩阵数量
二维数组matrix，不是0就是1. 返回内部全是1的子矩阵数量。

In [75]:
'''
最优解O(N^2)：
[[1,1,1,1,1]   index = 0
 [1,0,1,1,1]   index = 1
 [1,1,1,0,1]   index = 2
 [1,1,1,1,1]]  index = 3
从上至下
先求第0行做地基，求哪个子矩阵内部全是1有多少个。 [1,1,1,1,1], 这个list用单调栈处理一遍。
                                                      求得某一个位置x的左右最近比他小的位置i、j，可以计算出(j-i+1)=m范围都可以以x做高
                                                      那么以x为高的子矩阵有 m*(m+1)/2 个。 x-1..x-2..都是这个值。直到算到 x左右最近的值较大的那个(留着他自己算)
再求第1行做地基，求哪个子矩阵内部全是1有多少个。 [2,0,2,2,2]
再求第2行做地基，求哪个子矩阵内部全是1有多少个。 [3,1,3,0,3]
再求第3行做地基，求哪个子矩阵内部全是1有多少个。 [4,2,4,1,4], 一共N行，所以说N*N
'''

def sizeRectangle(arr):
    if not arr or not arr[0]:
        return 0
    N = len(arr)
    M = len(arr[0])
    base = [int(i) for i in arr[0]]
#     print("======== base 0 ========")
    ans = getRectSize(base)
    for i in range(1, N):
        base = getBase(base, arr[i])
#         print("======== base {} ========".format(i))
        ans += getRectSize(base)
    return ans
        
def getRectSize(arr):
    N = len(arr)
    stack = list()
    rangeList = [[None]*2 for _ in range(N)]
    for i in range(N):
        while stack and arr[stack[-1]] >= arr[i]:
            popIndex = stack.pop()
            if stack:
                rangeList[popIndex][0] = stack[-1]
            else:
                rangeList[popIndex][0] = -1
            rangeList[popIndex][1] = i
        stack.append(i)
    while stack:
        popIndex = stack.pop()
        if stack:
            rangeList[popIndex][0] = stack[-1]
        else:
            rangeList[popIndex][0] = -1
        rangeList[popIndex][1] = N
    ans = 0
    for i in range(N):
        rangeL,rangeR = rangeList[i][0], rangeList[i][1]
        left = 0
        right = 0
        if rangeL >= 0:
            left = arr[rangeL]
        if rangeR <= N - 1:
            right = arr[rangeR]
        maxSize = max(left, right)
        rangeLtoR = (rangeR - rangeL - 1)
        size = arr[i] - maxSize
#         print("{} x {} x {} = {}".format(size, leftSide, rightSide, size * leftSide * rightSide))
        ans += size * (rangeLtoR * (rangeLtoR+1) / 2)
            

    return ans
        
def getBase(base, arr):
    for i in range(len(base)):
        if int(arr[i]) == 0:
            base[i] = 0
        else:
            base[i] += int(arr[i])
    return base
            


In [74]:
# arr = [["1","0","1","0","0"],["1","0","1","1","1"],["1","1","1","1","1"],["1","0","0","1","0"]]
arr = [["1","0","1",],["1","1","1"],["1","1","0"]]

# arr = [["0","1"],["1","0"]]
sizeRectangle(arr)

======== base 0 ========
======== base 1 ========
======== base 2 ========


17.0